<a href="https://colab.research.google.com/github/GeorgeGPUGH/Machine-Learning/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Data Cleaning
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

df_ratings_filtered = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)
                                 & df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)]

# Merge the datasets
df_merged = pd.merge(df_ratings_filtered, df_books, on='isbn')


In [ ]:
# Data Transformation
# Creating a pivot table
book_matrix = df_merged.pivot_table(index='title', columns='user', values='rating').fillna(0)
book_sparse_matrix = csr_matrix(book_matrix.values)

In [ ]:
# Model Building
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5, n_jobs=-1)
model_knn.fit(book_sparse_matrix)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
   # Identify the index of the given book in the book matrix
    try:
        book_index = book_matrix.index.tolist().index(book)
    except ValueError:
        return []  # Return empty list if book not found

    # Use KNN model to find the nearest neighbors
    distances, indices = model_knn.kneighbors(book_matrix.iloc[book_index,:].values.reshape(1, -1), n_neighbors=6)

    # Extract and format the recommendations
    recommended_books = [[book_matrix.index[indices.flatten()[i]], float(distances.flatten()[i])] for i in range(2, 6)]

    return [book, recommended_books]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()